<a href="https://colab.research.google.com/github/NaotoKubota/Seimei-2021/blob/master/2021%E5%B9%B4%E5%BA%A6%E7%94%9F%E5%91%BD%E7%A7%91%E5%AD%A6%E7%A7%91%E9%9B%86%E4%B8%AD%E8%AC%9B%E7%BE%A9_RNAseq%E8%A7%A3%E6%9E%90%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%97%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2021年度生命科学科講義用スクリプト v1.1.0

1. Minicondaによるツールのインストール（seqtk, fastp, hisat2, samtools, subread）
2. RNA-seqデータのダウンロード
3. ダウンサンプリング（seqtk）
4. クオリティーコントロール（fastp）
5. マッピング（hisat2） & samファイルをbamファイルに変換（samtools）
6. 発現量のカウント（featureCounts）

2021.07.14 久保田

## 1. Minicondaによるツールのインストール（seqtk, fastp, hisat2, samtools, subread）

In [ ]:
%%bash
# Minicondaのダウンロード
wget https://repo.continuum.io/miniconda/Miniconda3-py38_4.8.3-Linux-x86_64.sh && \
bash Miniconda3-py38_4.8.3-Linux-x86_64.sh -bfp /usr/local

In [ ]:
%%bash
# 必要なツールのインストール
conda config --add channels conda-forge
conda config --add channels bioconda
conda install seqtk fastp hisat2 samtools subread 

## 2. RNA-seqデータのダウンロード

In [ ]:
%%bash
# RNA-seqデータのダウンロード
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR650/003/SRR6503273/SRR6503273_1.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR650/003/SRR6503273/SRR6503273_2.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR650/004/SRR6503274/SRR6503274_1.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR650/004/SRR6503274/SRR6503274_2.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR650/005/SRR6503275/SRR6503275_1.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR650/005/SRR6503275/SRR6503275_2.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR650/006/SRR6503276/SRR6503276_1.fastq.gz
wget ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR650/006/SRR6503276/SRR6503276_2.fastq.gz

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## 3. ダウンサンプリング（seqtk）

In [ ]:
%%bash

for SRR_ID in SRR650327{3..6};
do

  # 1000万リードをランダムサンプリング
  seqtk sample -s 100 ${SRR_ID}_1.fastq.gz 10000000 | gzip -c > ${SRR_ID}_1_10M.fastq.gz
  seqtk sample -s 100 ${SRR_ID}_2.fastq.gz 10000000 | gzip -c > ${SRR_ID}_2_10M.fastq.gz

  # 不要なファイルを削除
  rm ${SRR_ID}_1.fastq.gz ${SRR_ID}_2.fastq.gz

done

tcmalloc: large alloc 1120002048 bytes == 0x563114630000 @  0x7fa91f045001 0x563112af4277 0x7fa91e8a2bf7 0x563112aed630
tcmalloc: large alloc 1120002048 bytes == 0x560310f2c000 @  0x7f759a523001 0x56030e92e277 0x7f7599d80bf7 0x56030e927630
tcmalloc: large alloc 1120002048 bytes == 0x56348a97a000 @  0x7fdfe8940001 0x563489ec6277 0x7fdfe819dbf7 0x563489ebf630
tcmalloc: large alloc 1120002048 bytes == 0x560e9887c000 @  0x7f57b943e001 0x560e97e0a277 0x7f57b8c9bbf7 0x560e97e03630
tcmalloc: large alloc 1120002048 bytes == 0x55aea3656000 @  0x7f58621ed001 0x55aea29c5277 0x7f5861a4abf7 0x55aea29be630
tcmalloc: large alloc 1120002048 bytes == 0x563cb5922000 @  0x7f5c37db6001 0x563cb46b6277 0x7f5c37613bf7 0x563cb46af630
tcmalloc: large alloc 1120002048 bytes == 0x55d207784000 @  0x7f9176ec8001 0x55d2065b6277 0x7f9176725bf7 0x55d2065af630
tcmalloc: large alloc 1120002048 bytes == 0x55d671870000 @  0x7fa36c0fb001 0x55d670cfd277 0x7fa36b958bf7 0x55d670cf6630


## 4. クオリティーコントロール（fastp）

In [ ]:
%%bash

for SRR_ID in SRR650327{3..6};
do

  # fastpを使ってクオリティーコントロール
  # tail末端の1塩基をトリミングする。 20bp以下は捨てる。2スレッド使う。
  fastp -i ${SRR_ID}_1_10M.fastq.gz -I ${SRR_ID}_2_10M.fastq.gz \
  -o ${SRR_ID}_1_10M_filter.fastq.gz -O ${SRR_ID}_2_10M_filter.fastq.gz \
  -h ${SRR_ID}_report.html -j ${SRR_ID}_report.json -q 15 -n 10 -t 1 -T 1 -l 20 -w 2

  # 元のfastqファイルを削除
  rm ${SRR_ID}_1_10M.fastq.gz ${SRR_ID}_2_10M.fastq.gz

done

## 5. マッピング（hisat2） & samファイルをbamファイルに変換（samtools）

In [ ]:
%%bash
# hisat2のindexをダウンロード
wget ftp://ftp.ccb.jhu.edu/pub/infphilo/hisat2/data/hg38.tar.gz
tar -zxvf hg38.tar.gz
rm hg38.tar.gz 

hg38/
hg38/make_hg38.sh
hg38/genome.8.ht2
hg38/genome.5.ht2
hg38/genome.7.ht2
hg38/genome.6.ht2
hg38/genome.4.ht2
hg38/genome.3.ht2
hg38/genome.1.ht2
hg38/genome.2.ht2


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
%%bash

for SRR_ID in SRR650327{3..6};
do

  # hisat2によるマッピング
  hisat2 -t -p 2 -x hg38/genome \
  -1 ${SRR_ID}_1_10M_filter.fastq.gz -2 ${SRR_ID}_2_10M_filter.fastq.gz \
  -S ${SRR_ID}.sam --summary-file ${SRR_ID}.txt

  # samtoolsでsamをbamに変換 & sort
  samtools sort -@ 2 -O bam -o ${SRR_ID}.sort.bam ${SRR_ID}.sam

  # 不要なファイルの削除
  rm ${SRR_ID}_1_10M_filter.fastq.gz ${SRR_ID}_2_10M_filter.fastq.gz ${SRR_ID}.sam

done

In [ ]:
%%bash

# SRR6503273.sort.bamのchr22にマップされたリードのみ抽出
samtools index SRR6503273.sort.bam && \
samtools view -b SRR6503273.sort.bam chr22 > SRR6503273.chr22.sort.bam && \
samtools index SRR6503273.chr22.sort.bam

## 6. 発現量のカウント（featureCounts）

In [ ]:
%%bash
# GTFファイルのダウンロード
wget ftp://ftp.ensembl.org/pub/release-98/gtf/homo_sapiens/Homo_sapiens.GRCh38.98.gtf.gz
gzip -d Homo_sapiens.GRCh38.98.gtf.gz

In [ ]:
%%bash
# featureCountsで発現量をカウント
featureCounts -p -B -t exon -g gene_id \
-a Homo_sapiens.GRCh38.98.gtf \
-o all_counts.txt \
*.sort.bam


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.1

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 4 BAM files                                      ||
||                           o SRR6503273.sort.bam                            ||
||                           o SRR6503274.sort.bam                            ||
||                           o SRR6503275.sort.bam                            ||
||                           o SRR6503276.sort.bam               

`all_counts.txt`をローカルPCにダウンロードする